In [1]:
import numpy as np
import pprint
import sys
import warnings

sys.path.append('../src')
from simulation.user_sessions import simulate_users
from experiments.pre_test_analysis import required_sample_size, cuped_corr_coef
from experiments.experiment import experiment

In [2]:
cuped_corrcoef_list = []
for _ in range(5):
    cuped_corrcoef = cuped_corr_coef(100000, 4)
    cuped_corrcoef_list.append(cuped_corrcoef)

cuped_corrcoef_avg = np.mean(cuped_corrcoef_list)
print('Estimated CUPED correlation coefficient: {:.3f}'.format(cuped_corrcoef_avg))

pre_c_users, post_c_users, pre_t_users, post_t_users = simulate_users(100000,4,exp_effect=.03)
pre_exp_users_sum = np.vstack([pre_c_users, pre_t_users]).sum(axis=1)
post_exp_users_sum = np.vstack([post_c_users, post_t_users]).sum(axis=1)

t_test_n, t_test_cuped_n, seq_test_n, seq_test_cuped_n = required_sample_size(pre_exp_users_sum, cuped_corr_coef=cuped_corrcoef_avg, seq_final_alpha=.045)
print(f'T-test required sample size: {t_test_n}')
print(f'T-test with CUPED required sample size: {t_test_cuped_n}')
print(f'Group sequential test required sample size: {seq_test_n}')
print(f'Group sequential test with CUPED required sample size: {seq_test_cuped_n}')

Estimated CUPED correlation coefficient: 0.751
T-test required sample size: 20175
T-test with CUPED required sample size: 8806
Group sequential test required sample size: 20824
Group sequential test with CUPED required sample size: 9089


~.75 correlation coefficient between pre- and post-experiment sessions over a 4 week period seems reasonable. It creates a ~56% decrease in required sample size when using CUPED. Group sequential test is only slightly higher than t-test since I will be using the O'Brien-Flemming method.

In [3]:
exp = experiment(
    pre_c_user_sessions=pre_c_users,
    post_c_user_sessions=post_c_users,
    pre_t_user_sessions=pre_t_users,
    post_t_user_sessions=post_t_users,
    t_test_n=t_test_n,
    t_test_cuped_n=round(t_test_cuped_n * 1.1),
    seq_test_n=seq_test_n,
    seq_test_cuped_n=round(seq_test_cuped_n * 1.1),
)
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    exp.create_test_groups()
    exp.create_test_entries()
    exp.create_test_entry_data()
    exp.run_experiment()
    pprint.pprint(exp.results)

{'seq': {'p_value': 0.0, 'sample_size': 20824, 'significance': 1},
 'seq_cuped': {'p_value': 0.0, 'sample_size': 10000, 'significance': 1},
 'ttest': {'p_value': 0.0, 'sample_size': 40350, 'significance': 1},
 'ttest_cuped': {'p_value': 0.011, 'sample_size': 19374, 'significance': 1}}


Adding extra 10% to calculated minimum sample size for CUPED models to account for the larger estimation of variance.